In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Load and preprocess the dataset (as in your attached file)
data = pd.read_csv("job_applicant_data.csv")
data = data.drop(columns=['Unnamed: 0'])
data['NumTechWorkedWith'] = data['HaveWorkedWith'].str.count(';') + 1
data = data.drop(columns=['HaveWorkedWith'])

# Encode sensitive and other attributes
gender_mapping = {'Man': 1, 'Woman': 0, 'NonBinary': 2}
inv_gender_mapping = {v: k for k, v in gender_mapping.items()}
data['Gender'] = data['Gender'].map(gender_mapping)

age_mapping = {'>35': 1, '<35': 0}
inv_age_mapping = {v: k for k, v in age_mapping.items()}
data['Age'] = data['Age'].map(age_mapping)

ed_level_mapping = {'PhD': 4, 'Master': 3, 'Undergraduate': 2, 'NoHigherEd': 1, 'Other': 0}
inv_ed_level_mapping = {v: k for k, v in ed_level_mapping.items()}
data['EdLevel'] = data['EdLevel'].map(ed_level_mapping)

label_encoders = {}
for col in ['Age', 'Accessibility', 'MentalHealth', 'MainBranch', 'Country']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str))
    label_encoders[col] = le

In [ ]:
# Split the data into features, target, and sensitive attribute (Gender)
X = data.drop(columns=['Employed'])
y = data['Employed']
S = data["Gender"]

X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(
    X, y, S, test_size=0.2, random_state=42
)
X_train_W_Gender = X_train.copy()
X_test_W_Gender = X_test.copy()
# Remove Gender from training features
X_train = X_train.drop(columns=['Gender'])
X_test = X_test.drop(columns=['Gender'])

X_train_np = X_train.values
X_test_np = X_test.values
y_train = y_train.values.astype(float)
y_test = y_test.values.astype(float)
n_train = len(y_train)
n_test = len(y_test)

# Initialize predictor F (in log-odds space) as a constant function
p_mean = np.mean(y_train)
F_train = np.full(n_train, np.log(p_mean / (1 - p_mean)))
F_test  = np.full(n_test,  np.log(p_mean / (1 - p_mean)))

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Define the adversary network (a small neural network)
class Adversary(nn.Module):
    def __init__(self):
        super(Adversary, self).__init__()
        self.fc1 = nn.Linear(1, 16)   # Input: predictor's output (scalar)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 3)    # Output: logits for 3 classes (0,1,2)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No softmax here; CrossEntropyLoss applies it internally
        return x


In [ ]:

adv_model = Adversary()
adv_optimizer = optim.Adam(adv_model.parameters(), lr=0.01)
adv_criterion = nn.CrossEntropyLoss()

# Parameters for binning the combined residual u
num_bins = 10  # number of bins
# We will determine bin edges dynamically from u at each iteration

# Boosting loop with adversarial correction (min–max formulation) using RandomForestClassifier as weak learner
lambda_fairness = 0.015  # Trade-off parameter
M = 50                   # Number of boosting iterations
alpha = 1.0              # Step size for each iteration

In [ ]:
for m in range(M):
    # Compute current predicted probabilities using the sigmoid function
    p_pred = sigmoid(F_train)
    # Standard pseudoresiduals for logistic loss: r = y - p_pred
    r = y_train - p_pred

    # Compute adversary gradient: convert F_train to a torch tensor with gradients enabled
    F_tensor = torch.tensor(F_train.reshape(-1, 1), dtype=torch.float32, requires_grad=True)
    p_tensor = torch.sigmoid(F_tensor)
    adv_logits = adv_model(p_tensor)
    s_tensor = torch.tensor(S_train.values, dtype=torch.long)
    adv_loss = adv_criterion(adv_logits, s_tensor)
    adv_loss.backward()
    t = F_tensor.grad.detach().numpy().flatten()

    # Combine the gradients: u = r - lambda_fairness * t
    u = r - lambda_fairness * t

    # --- Using RandomForestClassifier as weak learner ---
    bin_edges = np.linspace(np.min(u), np.max(u), num_bins+1)
    # Digitize u: bin indices from 1 to num_bins; subtract 1 to have indices 0..num_bins-1
    u_bins = np.digitize(u, bin_edges) - 1
    # For regression approximation, we take the midpoint of each bin as the representative value.
    bin_midpoints = (bin_edges[:-1] + bin_edges[1:]) / 2.0

    # Train a RandomForestClassifier to predict the bin index from X_train_np.
    rf_weak = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=42)
    rf_weak.fit(X_train_np, u_bins)
    # Predict bin indices for training and test sets.
    u_bins_pred_train = rf_weak.predict(X_train_np)
    u_bins_pred_test = rf_weak.predict(X_test_np)
    # Convert predicted bin indices to continuous values using the midpoints.
    h_train = np.array([bin_midpoints[i] for i in u_bins_pred_train])
    h_test = np.array([bin_midpoints[i] for i in u_bins_pred_test])

    # Update the predictor F by adding the scaled weak learner prediction
    F_train = F_train + alpha * h_train
    F_test  = F_test  + alpha * h_test

    # Update the adversary network using the new predictor outputs
    F_tensor_new = torch.tensor(F_train.reshape(-1, 1), dtype=torch.float32)
    p_tensor_new = torch.sigmoid(F_tensor_new)
    adv_logits_new = adv_model(p_tensor_new)
    adv_loss_new = adv_criterion(adv_logits_new, s_tensor)
    adv_optimizer.zero_grad()
    adv_loss_new.backward()
    adv_optimizer.step()

    # Monitoring: compute and print the predictor loss on the training set
    pred_loss = -np.mean(y_train * np.log(p_pred + 1e-8) + (1 - y_train) * np.log(1 - p_pred + 1e-8))
    print(f"Iteration {m}: Predictor Loss = {pred_loss:.4f}, Adversary Loss = {adv_loss_new.item():.4f}")

In [ ]:
# Final predictions and evaluation on the test set
final_prob_test = sigmoid(F_test)
final_pred_test = (final_prob_test > 0.5).astype(int)
test_acc = accuracy_score(y_test, final_pred_test)
print("\nFinal Test Accuracy:", test_acc)


In [ ]:
# Fairness metrics calculation
def compute_demographic_parity(predictions, sensitive, group):
    idx = (sensitive == group)
    return np.mean(predictions[idx])

print("\nDemographic Parity on Test Set:")
for group in np.unique(S_test.values):
    rate = compute_demographic_parity(final_pred_test, S_test.values, group)
    print(f"Group {group} ({inv_gender_mapping[group]}): Positive Rate = {rate:.4f}")

print("\nEqual Opportunity (True Positive Rate) on Test Set:")
for group in np.unique(S_test.values):
    group_idx = (S_test.values == group)
    positive_idx = (y_test == 1)
    group_positive = group_idx & positive_idx
    if np.sum(group_positive) > 0:
        tpr = np.sum(final_pred_test[group_positive] == 1) / np.sum(group_positive)
        print(f"Group {group} ({inv_gender_mapping[group]}): TPR = {tpr:.4f}")
    else:
        print(f"Group {group} ({inv_gender_mapping[group]}): No positive samples")